In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [12]:
from src.inference import get_feature_store

In [13]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*28)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 23:05:45,433 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 23:05:45,443 INFO: Initializing external client
2025-03-03 23:05:45,445 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 23:05:46,416 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Fetching data from 2025-02-04 04:05:45.433813+00:00 to 2025-03-04 03:05:45.433813+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.37s) 


In [14]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides


In [15]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_hour         0 non-null      datetime64[us, Etc/UTC]
 1   pickup_location_id  0 non-null      int32                  
 2   rides               0 non-null      int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 132.0 bytes


In [16]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [17]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pickup_hour         0 non-null      datetime64[us]
 1   pickup_location_id  0 non-null      int32         
 2   rides               0 non-null      int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 132.0 bytes


In [19]:
from src.data_utils import transform_ts_data_info_features_and_target
features = transform_ts_data_info_features_and_target(ts_data, window_size=24*27, step_size=23)

ValueError: No data could be transformed. Check if input DataFrame is empty or window size is too large.

In [ ]:
features

In [ ]:
from src.inference import load_batch_of_features_from_store

In [ ]:
current_date

In [ ]:
features=load_batch_of_features_from_store(current_date)
features

In [ ]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

In [ ]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [ ]:
predictions

In [ ]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values